Tiempos hasta Install
=====================

Se desea armar un set de entrenamiento con el tiempo que pasa hasta que un un dispositivo convierta. 

Sabiendo que el set de datos posee informacion del dia 18 al 26 

Para esto se arman ventanas de los siguientes dias: 

1. 21 al 23
2. 22 al 24
3. 23 al 25
4. 24 al 26

Y se desea obtener el minimo de tiempo en segundos hasta que instala cada dispositivo.


Luego, se cruza los labels armados con los labels a predecir y se  asigna un valor por defecto para aquellos que no se encuentran en alguna ventana.

Opciones: 

- Maximo tiempo posible: Tiempo de ventana
- Maximo tiempo visto: Maximo tiempo hasta install de alguna ventana
- Media de tiempo entre installs vistos

Para los ultimos 2 hay que definir que hacer si no se instalo ninguna vez. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mp
import datetime as dt
import gzip

%matplotlib inline

In [2]:
installs_dtypes = {
    'application_id': np.int32,
    'ref_type': np.int64,
    'ref_hash': np.int64, 
    'click_hash':'category',
    'attributed': 'category',
    'implicit': 'category',
    #'device_countrycode': 'object', 
    'device_brand': 'category',
    'device_model': 'category', 
    'session_user_agent': 'category', 
    'user_agent': 'category', 
    'event_uuid':'object',
    'kind': 'category',
    'wifi': 'category', 
    'trans_id': 'object', 
    #'ip_address':'object', 
    'device_language': 'category'
}
install_cols = list(installs_dtypes.keys()) + ['created']
installs = pd.read_csv('../data/installs.csv.gzip', 
                       compression='gzip', 
                       usecols=install_cols,
                       dtype= installs_dtypes,
                      parse_dates=['created'])

to_predict = pd.read_csv('../data/target_final_competencia_revamped.csv')
to_predict = to_predict.apply(lambda x: np.int64(x['ref_hash'][0:x['ref_hash'].find('_')]), axis='columns').drop_duplicates().to_frame()
to_predict.columns = ['ref_hash']

In [3]:
all_ref_hash = installs['ref_hash'].drop_duplicates().to_frame()
all_ref_hash.columns = ['ref_hash']

In [4]:
all_ref_hash.append(to_predict).drop_duplicates().count()

ref_hash    396509
dtype: int64

In [5]:
window = 3
j=1
installs_training = pd.DataFrame()

for i in range(21,25): #[19, 20, 21, 22, 23, 24]
    print("Ventana: ", j)
    temp = pd.DataFrame()
    
    temp[['ref_hash','date_install']] = installs.loc[(installs['created'].dt.day >= i) & (installs['created'].dt.day < (i+window))].groupby('ref_hash', as_index=False)['created'].min()
    temp['window_nr'] = j
    temp['window_date_start'] = dt.datetime(2019,4,i)
    temp['secs_to_install'] = (temp['date_install']-temp['window_date_start']).transform(lambda x: x.total_seconds())
    print("  Encontrados: ", len(temp))
    installs_training = installs_training.append(temp,sort=True)
    
    #Agrego los que no se encontraron en esta ventana pero si estan en installs (aparecen en alguna)
    temp = all_ref_hash.merge(installs_training.loc[installs_training['window_nr'] == j], how='left',on='ref_hash')
    temp = temp.loc[temp['window_nr'].isnull()]
    temp['window_nr'] = j
    temp['window_date_start'] = dt.datetime(2019,4,i)
    temp['date_install'] = dt.datetime(2019,4,i+window)
    temp['secs_to_install'] = (temp['date_install']-temp['window_date_start']).transform(lambda x: x.total_seconds())
    print("  Agregados sin installs en esta ventana: ", len(temp))
    installs_training = installs_training.append(temp[['ref_hash','window_nr','window_date_start','date_install','secs_to_install']],sort=True)
    
    #Agrego los que no se encontraron en install pero se deben predecir conversiones
    temp = to_predict.merge(installs_training.loc[installs_training['window_nr'] == j], how='left', on='ref_hash')
    temp = temp.loc[temp['window_nr'].isnull()]
    print("  Agregados sin installs pero en lista a predecir: ", len(temp))
    temp['window_nr'] = j
    temp['window_date_start'] = dt.datetime(2019,4,i)
    temp['date_install'] = dt.datetime(2019,4,i+window)
    temp['secs_to_install'] = (temp['date_install']-temp['window_date_start']).transform(lambda x: x.total_seconds())
    installs_training = installs_training.append(temp,sort=True)
    
    j = j+1

sorted_installs = installs_training.sort_values(['ref_hash','window_nr'])[['ref_hash','window_nr','window_date_start','date_install','secs_to_install']]

Ventana:  1
  Encontrados:  133278
  Agregados sin installs en esta ventana:  260287
  Agregados sin installs pero en lista a predecir:  2944
Ventana:  2
  Encontrados:  132834
  Agregados sin installs en esta ventana:  260731
  Agregados sin installs pero en lista a predecir:  2944
Ventana:  3
  Encontrados:  133885
  Agregados sin installs en esta ventana:  259680
  Agregados sin installs pero en lista a predecir:  2944
Ventana:  4
  Encontrados:  138317
  Agregados sin installs en esta ventana:  255248
  Agregados sin installs pero en lista a predecir:  2944


In [6]:
sorted_installs['window_nr'].value_counts()

4    396509
3    396509
2    396509
1    396509
Name: window_nr, dtype: int64

In [7]:
sorted_installs.head(20)

,ref_hash,window_nr,window_date_start,date_install,secs_to_install
0,40621409780134,1,2019-04-21,2019-04-21 19:17:47.657,69467.657
41317,40621409780134,2,2019-04-22,2019-04-25 00:00:00.000,259200.000
41317,40621409780134,3,2019-04-23,2019-04-26 00:00:00.000,259200.000
41317,40621409780134,4,2019-04-24,2019-04-27 00:00:00.000,259200.000
100266,41863526108385,1,2019-04-21,2019-04-24 00:00:00.000,259200.000
100266,41863526108385,2,2019-04-22,2019-04-25 00:00:00.000,259200.000
100266,41863526108385,3,2019-04-23,2019-04-26 00:00:00.000,259200.000
100266,41863526108385,4,2019-04-24,2019-04-27 00:00:00.000,259200.000
182806,90072729247980,1,2019-04-21,2019-04-24 00:00:00.000,259200.000
0,90072729247980,2,2019-04-22,2019-04-24 18:30:50.199,239450.199


In [9]:
sorted_installs.to_csv('installs_training.csv',index=False)